In [1]:
import requests

import pandas as pd
import numpy as np

from collections import defaultdict

import time

In [2]:
API_KEY = '***'

In [3]:
base_path = 'https://cryptopanic.com/api'
posts_path = f'/v1/posts/'

In [4]:
resp = requests.get(f'{base_path}{posts_path}', params={
    'auth_token': API_KEY,
    'public': True,
    'kind': 'news',
#     'filter': 'important,rising',
    'currencies': 'BTC,ETH,USDT,DOGE,BNB,XRP,MATIC,DOT,TON,NEAR,TWT,SFP,CHZ',
    'metadata': True,
})

In [5]:
out_json = resp.json()

In [6]:
out_json

{'count': 200,
 'next': 'https://cryptopanic.com/api/v1/posts/?auth_token=f9811e03c04f7695df2fe00893a072cf66607e76&public=True&kind=news&currencies=BTC%2CETH%2CUSDT%2CDOGE%2CBNB%2CXRP%2CMATIC%2CDOT%2CTON%2CNEAR%2CTWT%2CSFP%2CCHZ&metadata=True&page=2',
 'previous': None,
 'results': [{'kind': 'news',
   'domain': 'ambcrypto.com',
   'source': {'title': 'AMBCrypto',
    'region': 'en',
    'domain': 'ambcrypto.com',
    'path': None},
   'title': 'Could BNB’s diminishing enthusiasm have investors rethink their 2023 investments',
   'published_at': '2022-12-26T01:30:59Z',
   'slug': 'Could-BNBs-diminishing-enthusiasm-have-investors-rethink-their-2023-investments',
   'currencies': [{'code': 'BNB',
     'title': 'BNB',
     'slug': 'binancecoin',
     'url': 'https://cryptopanic.com/news/binancecoin/'}],
   'id': 17238555,
   'url': 'https://cryptopanic.com/news/17238555/Could-BNBs-diminishing-enthusiasm-have-investors-rethink-their-2023-investments',
   'created_at': '2022-12-26T01:30:59Z

In [7]:
total_count = 0

In [8]:
data_dict = defaultdict(list)

In [9]:
while total_count < 1000:
    print('--> News downloaded', out_json['count'])
    total_count += out_json['count'] / 10
    
    print(len(out_json['results']))
    for res in out_json['results']:
        data_dict['source'].append(res['source']['title'])
        data_dict['title'].append(res['title'])
        data_dict['published_at'].append(res['published_at'])
        data_dict['currencies'].append(','.join([cur['code'] for cur in res['currencies']]))
        data_dict['full_news_url'].append(res['url'])
        if 'metadata' in res:
            data_dict['description'].append(res['metadata']['description'])
        else:
            data_dict['description'].append('')
    try:
        resp = requests.get(out_json['next'])
        out_json = resp.json()
        retry = False
    except Exception as e:
        print(e)
        break
    time.sleep(1)

--> News downloaded 200
20
--> News downloaded 200
20
--> News downloaded 200
20
--> News downloaded 200
20
--> News downloaded 200
20
--> News downloaded 200
20
--> News downloaded 200
20
--> News downloaded 200
20
--> News downloaded 200
20
--> News downloaded 200
20
Invalid URL 'None': No scheme supplied. Perhaps you meant http://None?


In [10]:
df = pd.DataFrame(data_dict)
df

,source,title,published_at,currencies,full_news_url,description
0,AMBCrypto,Could BNB’s diminishing enthusiasm have invest...,2022-12-26T01:30:59Z,BNB,https://cryptopanic.com/news/17238555/Could-BN...,BNB released its weekly stats and December sta...
1,Bitcoin.com,"Mark Cuban: Bitcoin Is a Good Investment, Gold...",2022-12-26T01:30:22Z,BTC,https://cryptopanic.com/news/17238556/Mark-Cub...,Shark Tank star and the owner of the NBA team ...
2,Feed - Cryptopotato.Com,"Institutions Are Not Interested in Crypto, Say...",2022-12-26T01:22:09Z,BTC,https://cryptopanic.com/news/17238548/Institut...,<p>Institutional investors feel relieved they ...
3,Forkast.News,"Markets: Bitcoin, Ether little changed; Dogeco...",2022-12-26T01:20:27Z,"BTC,ETH,DOGE",https://cryptopanic.com/news/17238549/Markets-...,Bitcoin and Ether changed little on Monday mor...
4,CEO of Metal,RT @Oonabot: Merry Christmas my #Dogecoin #Dog...,2022-12-26T00:29:11Z,"MTL,DOGE",https://cryptopanic.com/news/17238499/RT-Oonab...,
...,...,...,...,...,...,...
195,Feed - Cryptopotato.Com,"OKX Proof-of-Reserves Report: Bitcoin, Ether, ...",2022-12-23T18:10:53Z,USDT,https://cryptopanic.com/news/17225884/OKX-Proo...,<p>The recently published Merkle tree PoR prog...
196,CryptoSlate,Bitcoin miners increased hash rate and product...,2022-12-23T18:00:30Z,BTC,https://cryptopanic.com/news/17225863/Bitcoin-...,<p>The price of Bitcoin (BTC) fell by 64.68% t...
197,Co-founder of Ethereum,Looking forward to seeing who the next CEO is ...,2022-12-23T17:57:19Z,ETH,https://cryptopanic.com/news/17225833/Looking-...,
198,The Daily Hodl,Twitter Adds Bitcoin (BTC) and Ethereum (ETH) ...,2022-12-23T17:55:29Z,"BTC,ETH",https://cryptopanic.com/news/17225843/Twitter-...,<p>Social media giant Twitter is adding Bitcoi...


In [11]:
df.to_csv('news-8.csv', index=False)